In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# 1) 모델 파일(.h5) 경로 지정
model_path = 'best_model.h5'  # 또는 'final_fine_tuned_model.h5'

# 2) 테스트할 이미지 파일 경로 지정
img_path = "C:\\Users\\USER\\desktop\\크롤링사진_03.jpg"  # 실제 파일 위치로 바꿔주세요

# 3) 클래스 이름 목록 (train_dir 하위폴더 순서와 동일하게)
#    예를 들어, train_dir = "dataset/train" 안에 "heavy", "medium", "clean" 폴더가 있다면:
class_names = ['heavy', 'medium', 'clean']

# ------------------------------------------------------------------
# 4) 1단계: 모델 불러오기
# ------------------------------------------------------------------
print("1) 모델 불러오는 중...")
try:
    model = tf.keras.models.load_model(model_path)
    print(f"✅ 모델을 '{model_path}'에서 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"❌ 모델을 불러오는 데 실패했습니다. 경로와 파일 이름을 확인하세요. 에러 내용:\n{e}")
    raise SystemExit("모델 로드 실패: 스크립트를 종료합니다.")


# ------------------------------------------------------------------
# 5) 2단계: 이미지 불러오기 및 전처리
# ------------------------------------------------------------------
print("\n2) 이미지를 불러와서 전처리하는 중...")
# (1) PIL 형식으로 이미지 읽기 (RGB 채널)
img = image.load_img(img_path, target_size=(224, 224))
#     -> model 학습 때 IMG_SIZE = (224, 224)로 맞췄으므로 여기서도 동일하게 맞춰줘야 합니다.

# (2) 넘파이 배열로 변환
img_array = image.img_to_array(img)
#     -> shape: (224, 224, 3), dtype: float32. 각 픽셀은 0~255 사이 값.

# (3) 정규화 (ResNet50 학습 시에도 1./255. 스케일을 적용했으므로 똑같이 해 줍니다.)
img_array = img_array / 255.0
#     -> shape: (224, 224, 3), dtype: float32, 값 범위 0~1로 바뀜.

# (4) 배치 차원 추가
#     -> 모델은 (batch_size, 224, 224, 3) 형태를 기대합니다. 배치 크기를 1로 만들어 줍니다.
img_batch = np.expand_dims(img_array, axis=0)
#     -> 최종 shape: (1, 224, 224, 3)

print("   • 원본 이미지 크기:", img.size, "(PIL 포맷)")  
print("   • 넘파이 배열로 변환 후 shape(정규화 전):", img_array.shape,   
      "→ 정규화 후에도 shape는 같고 dtype만 바뀜")
print("   • 배치 차원 추가 후 shape:", img_batch.shape)


# ------------------------------------------------------------------
# 6) 3단계: 모델로 예측(Predict)
# ------------------------------------------------------------------
print("\n3) 모델에 입력하여 예측(추론) 중...")
# model.predict는 numpy array를 받아서 확률 벡터를 반환합니다.
predictions = model.predict(img_batch)  # shape: (1, 3)  → [[p_heavy, p_medium, p_clean]]

# predictions[0]은 1차원 배열로 [p_heavy, p_medium, p_clean]
pred = predictions[0]

# (옵션) 퍼센트로 바꾸기 위해 100을 곱함
pred_percent = pred * 100  

# 가장 높은 확률의 인덱스
top_index = np.argmax(pred)  # 예: 0이면 'heavy', 1이면 'medium', 2이면 'clean'
top_class  = class_names[top_index]
top_prob   = pred[top_index] * 100.0

# ------------------------------------------------------------------
# 7) 4단계: 결과 출력
# ------------------------------------------------------------------
print("\n4) 예측 결과:")
for i, class_name in enumerate(class_names):
    print(f"   • {class_name:6s} : {pred_percent[i]:6.2f}%")
print(f"\n▶ 최종 판정: '{top_class}' 클래스 (확률: {top_prob:.2f}%)")

# 끝: 화면에 각 클래스별 확률과 최종 예측을 출력합니다.


1) 모델 불러오는 중...


✅ 모델을 'best_model.h5'에서 성공적으로 불러왔습니다.

2) 이미지를 불러와서 전처리하는 중...
   • 원본 이미지 크기: (224, 224) (PIL 포맷)
   • 넘파이 배열로 변환 후 shape(정규화 전): (224, 224, 3) → 정규화 후에도 shape는 같고 dtype만 바뀜
   • 배치 차원 추가 후 shape: (1, 224, 224, 3)

3) 모델에 입력하여 예측(추론) 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

4) 예측 결과:
   • heavy  :  36.95%
   • medium :  30.61%
   • clean  :  32.44%

▶ 최종 판정: 'heavy' 클래스 (확률: 36.95%)
